In [364]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import pandas as pd
from pandas_ply import *
install_ply(pd)

In [340]:
os.getcwd()
os.chdir('C:\Users\dylan_000\Downloads\Project')

#1. Population Data
###Let us read in the data on population counts. We only need the city name, the state, and the population for each of the 100 most populous locales.

In [432]:
populations = pd.read_csv('populations_raw.csv')
populations = populations[['GC_RANK.display-label.1','respop72014']]
populations['City'],populations['State'] = [city.split(',')[0] for city in populations['GC_RANK.display-label.1']],\
    [city.split(',')[1] for city in populations['GC_RANK.display-label.1']]
populations = populations.iloc[:,1:4]
populations.columns = ['Population','City','State']
populations = populations.iloc[:100,:]
populations.State = [state.lstrip().title() for state in populations.State]
populations.head()

,Population,City,State
0,8491079,New York city,New York
1,3928864,Los Angeles city,California
2,2722389,Chicago city,Illinois
3,2239558,Houston city,Texas
4,1560297,Philadelphia city,Pennsylvania


#2. Agency Employee Data
###Now it is time to examine the data on staffing levels for each agency. We will clean up the column names, and clean up the column for stata names in order to have a value in each row. 

In [433]:
employees = pd.read_excel('employees_raw.xls')
employees = employees[3:]
employees.columns = ['State','City','Population','TotalEmployees','TotalOfficers','TotalCivilians']
employees.State = [str(state).lower().title() for state in employees.State]
employees.head()
state is None
employees.index = range(len(employees))
for i in range(len(employees)):
    if state is None:
        state = 'Alabama'
        continue
    if employees.loc[i,'State'] == 'Nan':
        employees.loc[i,'State'] = state
    else: state = employees.loc[i,'State']
employees.head()

,State,City,Population,TotalEmployees,TotalOfficers,TotalCivilians
0,Alabama,Albertville,21655,69,44,25
1,Alabama,Andalusia,9098,39,28,11
2,Alabama,Anniston,22567,102,83,19
3,Alabama,Ardmore,1332,10,7,3
4,Alabama,Argo,4200,8,8,0


#3. Crime Data
###Last, but surely not least, we will bring the crime data into play for the year 2014. This is for each city-level agency, and this will need to be thoroughly cleaned before being exported to be analyzed in R.

In [555]:
crimes = pd.read_excel('crimes_raw.xls')
crimes = crimes[3:]
crimes.columns = ['State','City','Population','ViolentCrime','Murder','RapeRevised','RapeLegacy','Robbery','AggAssaults',\
                  'PropertyCrime','Burglary','Larceny','MVLarceny','Arson']
crimes.index = range(len(crimes))
crimes.State = [str(state).lower().title() for state in crimes.State]
state = None
for i in range(len(crimes)):
    if state is None:
        state = 'Alabama'
        continue
    if crimes.loc[i,'State'] == 'Nan':
        crimes.loc[i,'State'] = state
    else: state = crimes.loc[i,'State']
crimes['TotalRate'] = ((crimes.ViolentCrime + crimes.PropertyCrime)/crimes.Population) * 100000
crimes['ViolentRate'] = crimes.ViolentCrime/crimes.Population * 100000
crimes['PropertyRate'] = crimes.PropertyCrime/crimes.Population * 100000

NameError: name 'State' is not defined

In [475]:
welfare = pd.read_excel('welfare_raw.xlsx')
welfare.head()
welfare.State = [re.findall('[^0-9]', str(state).lower()) for state in welfare.State]
welfare.State = [''.join(state).title() for state in welfare.State]


In [541]:
joined = pd.merge(populations,welfare,left_on='State',right_on='State')
joined.City = [re.findall('(.+) city',city) for city in joined.City]
joined.City = [''.join(city) for city in joined.City]

In [553]:
crimes.City = [re.findall('(.+) city',str(city)) for city in crimes.City]

In [499]:
joined = pd.merge(joined,crimes[['State','City','ViolentRate','PropertyRate','TotalRate']],how='left',on='State')

In [554]:
crimes

,State,City,Population,ViolentCrime,Murder,RapeRevised,RapeLegacy,Robbery,AggAssaults,PropertyCrime,Burglary,Larceny,MVLarceny,Arson,TotalRate,ViolentRate,PropertyRate
0,Alabama,,2643,8,0,1,NaN,0,7,85,22,52,11,NaN,3518.729,302.6863,3216.042
1,Alabama,,4430,30,0,5,NaN,6,19,373,43,321,9,NaN,9097.065,677.2009,8419.865
2,Alabama,,747,1,0,0,NaN,0,1,31,6,22,3,NaN,4283.802,133.8688,4149.933
3,Alabama,,31568,58,1,9,NaN,11,37,612,68,526,18,NaN,2122.402,183.7304,1938.672
4,Alabama,,21655,23,1,5,NaN,8,9,756,230,454,72,NaN,3597.322,106.211,3491.111
5,Alabama,,14887,81,1,8,NaN,8,64,716,165,523,28,NaN,5353.664,544.0989,4809.565
6,Alabama,,2403,10,0,0,NaN,0,10,43,9,32,2,NaN,2205.576,416.1465,1789.43
7,Alabama,,22567,536,4,42,NaN,77,413,1711,650,969,92,NaN,9957.017,2375.15,7581.867
8,Alabama,,8369,46,0,13,NaN,3,30,567,140,382,45,NaN,7324.65,549.6475,6775.003
9,Alabama,,1332,2,0,0,NaN,0,2,22,2,19,1,NaN,1801.802,150.1502,1651.652


In [522]:
joined2.State = [re.findall('[^0-9]',state) for state in joined2.State]
joined2.State = [''.join(state) for state in joined2.State]

In [540]:
joined

,Population,City_x,State,Welfare,City_y,ViolentRate,PropertyRate,TotalRate
0,8491079,New York,New York,789,Adams Village,0,594.2734,594.2734
1,8491079,New York,New York,789,Addison Town and Village,77.88162,1908.1,1985.981
2,8491079,New York,New York,789,Afton Village6,0,121.9512,121.9512
3,8491079,New York,New York,789,Akron Village,35.18649,598.1703,633.3568
4,8491079,New York,New York,789,Albany6,813.4287,3943.405,4756.834
5,8491079,New York,New York,789,Albion Village6,442.7793,3474.114,3916.894
6,8491079,New York,New York,789,Alexandria Bay Village6,0,632.3397,632.3397
7,8491079,New York,New York,789,Alfred Village6,272.8175,744.0476,1016.865
8,8491079,New York,New York,789,Altamont Village6,58.03831,116.0766,174.1149
9,8491079,New York,New York,789,Amherst Town6,107.6897,1738.179,1845.869
